# GANs

GAN代表Generative Adversarial Nets，由Ian Goodfellow发明。 我们的概念是，我们将同时训练两个模型：发电机和评论家。 生成器将尝试制作类似于我们的数据集中的新图像，并且评论家的工作将尝试对来自发生器所做的假图像的真实图像进行分类。 生成器返回图像，鉴别器返回特征映射（根据输入大小，它可以是单个数字）。 通常情况下，鉴别器将被训练为在任何地方返回虚假图像和1.到处都是真实的图像。

该模块包含创建GAN所需的所有功能。

我们在每一步（或多或少）的意义上训练他们互相对抗，我们：

1. 冻结发电机并通过以下方式训练鉴别器：
    * 得到一批真实的图像（让我们称之为真实的）
    * 生成一批假图像（让我们称之为假）
    * 让鉴别器评估每批并从中计算损失函数; 重要的是，它积极地奖励真实图像的检测并惩罚假的图像
    * 用这种损失的梯度更新鉴别器的权重
2. 冻结鉴别器并通过以下方式训练发电机一步：
    * 生成一批假图像
    * 评估它上面的鉴别器
    * 返回一种损失，积极地奖励那些认为那些是真实形象的鉴别者; 重要的是，它积极地奖励真实图像的检测并惩罚假的图像
    * 用这种损失的梯度更新发电机的重量

---

## class GANLearner

`GANLearner(data:DataBunch, generator:Module, critic:Module, gen_loss_func:LossFunction, crit_loss_func:LossFunction, switcher:Callback=None, gen_first:bool=False, switch_eval:bool=True, show_img:bool=True, clip:float=None, **learn_kwargs) :: Learner`

适合GAN的学习者。

这是创建GAN的一般构造函数，您可能希望使用一种更易于使用的工厂方法。 从数据，生成器和评论家创建GAN。 数据应具有生成器期望的输入和想要作为目标的图像。

gen_loss_func是将应用于生成器的损失函数。 它需要三个参数fake_pred，target，output并且应该返回一个等级0张量。 output是生成器应用于输入的结果（批处理的xs），target是批处理的ys，fake_pred是鉴别器被给出输出的结果。 输出和目标可用于为GAN损失（像素丢失，特征丢失）添加特定损失，并且对于gan的良好训练，损失应该鼓励fake_pred尽可能接近1（发生器训练为傻瓜） 评论家）。

crit_loss_func是将应用于评论家的损失函数。 它需要两个参数real_pred和fake_pred。 real_pred是对目标图像（批量的ys）的批评的结果，而fake_pred是批评者应用于批量生成的批量生成的一批假的结果。

切换器是一个回调，应该告诉GAN何时从批评者切换到生成器，反之亦然。 默认情况下，它会对发生器的1次迭代进行5次迭代。 如果gen_first = True，模型将使用生成器开始训练。 如果switch_eval = True，则未训练的模型将在eval模式下打开（否则将在训练模式中保留，这意味着会更新某些统计信息，例如batchnorm图层中的运行平均值，或应用丢失）。

如果想要剪切权重，则应将剪辑设置为某个值（例如，参见Wassertein GAN）。

如果show_img = True，则由GAN生成的一个图像显示在每个纪元的末尾。

### Factory methods
**from_learners**

`from_learners(learn_gen:Learner, learn_crit:Learner, switcher:Callback=None, weights_gen:Point=None, **learn_kwargs)`

从learn_gen和learn_crit创建一个GAN。

直接从两个学习者创建一个GANLearner：一个用于生成器，一个用于评论者。 切换器和所有kwargs将传递给GANLearner的初始化以及以下丢失函数：

* loss_func_crit是应用于real_pred的learn_crit.loss_func的平均值，以及应用于fake_pred的learn_crit.loss_func和零的目标的目标
* loss_func_gen是应用于fake_pred的learn_crit.loss_func的平均值，以及应用于输出和目标的learn_gen.loss_func的一个目标（完整的鉴别器）。 每个贡献的权重可以在weights_gen中传递（默认值为1.和1）

**wgan**

`wgan(data:DataBunch, generator:Module, critic:Module, switcher:Callback=None, clip:float=0.01, **learn_kwargs)`

从数据，生成器和评论家创建WGAN。

Wasserstein GAN详见[本文]。 切换器和kwargs将传递给GANLearner init，剪辑重量剪辑。

---

## Switchers

在任何GAN培训中，您都需要告诉学习者何时从发生器切换到批评者，反之亦然。 以下两个回调是帮助您的示例。

像往常一样，不要直接调用on_something方法，fastai库会在训练期间为你做。

**class FixedGANSwitcher**

`FixedGANSwitcher(learn:Learner, n_crit:Union[int, Callable]=1, n_gen:Union[int, Callable]=1) :: LearnerCallback`

切换器做n_crit迭代的批评然后n_gen迭代的生成器。

**on_train_begin**

`on_train_begin(**kwargs)`

启动迭代计数。

**on_batch_end**

`on_batch_end(iteration, **kwargs)`

必要时切换模型。

**class AdaptiveGANSwitcher**

`AdaptiveGANSwitcher(learn:Learner, gen_thresh:float=None, critic_thresh:float=None) :: LearnerCallback`

当损失低于gen_thresh / crit_thresh时，切换回发电机/评论员的切换器。

**on_batch_end**

`on_batch_end(last_loss, **kwargs)`

必要时切换模型。

---

## 判别性LR
如果你想以与发电机不同的学习速度训练你的评论家，这将让你自动完成（即使你有学习率表）。

**class GANDiscriminativeLR**

`GANDiscriminativeLR(learn:Learner, mult_lr:float=5.0) :: LearnerCallback`

处理批评者处理乘以mult_lr的学习率的回调。

**on_batch_begin**

`on_batch_begin(train, **kwargs)`

如有必要，将当前的lr相乘。

**on_step_end**

`on_step_end(**kwargs)`

如有必要，将LR恢复到其值。

---

## 具体模型

**basic_critic**

`basic_critic(in_size:int, n_channels:int, n_features:int=64, n_extra_layers:int=0, **conv_kwargs)`

图像的基本评论n_channels x in_size x in_size。

该模型包含从n_channels到n_features的步幅2的第一个4乘4卷积层，接着是n_extra_layers 3乘3步幅1的卷积层。然后我们将多个4乘4步幅层的步幅2与多个特征乘以2 在每个阶段，以便in_size变为1. kwargs可用于自定义卷积层并传递给conv_layer。

**basic_generator**

`basic_generator(in_size:int, n_channels:int, noise_sz:int=100, n_features:int=64, n_extra_layers=0, **conv_kwargs)`

从noise_sz到图像的基本生成器n_channels x in_size x in_size。

该模型包含从noise_size到相应评论家的最后几个特征的步幅1的第一个4乘4转置卷积层。 然后我们在每个阶段放置尽可能多的4乘4转置的步幅2的卷积层，其中在每个阶段将多个特征除以2，使得图像最终具有高度并且具有in_size // 2。 最后，我们将3乘3的旋转层叠加到第3层，最后一层是4乘4的转置卷积，然后是步幅2，然后是tanh。 kwargs可用于自定义卷积层并传递给conv_layer。

**gan_critic**

`gan_critic(n_channels:int=3, nf:int=128, n_blocks:int=3, p:int=0.15)`

批评训练GAN。

---

## class GANTrainer

`GANTrainer(learn:Learner, switch_eval:bool=False, clip:float=None, beta:float=0.98, gen_first:bool=False, show_img:bool=True) :: LearnerCallback`

处理GAN培训。

LearnerCallback将负责处理两个不同的优化器（一个用于生成器，一个用于批评者），并在幕后完成所有工作，以便生成器（或批评者）处于训练模式，每次都有参数需求梯度 我们切换。

switch_eval = True意味着GANTrainer会将未训练的模型放入eval模式（如果它为False，则会更新其在batchnorm图层中的运行统计信息并将应用dropout）。 clip是应用于权重的剪辑（如果不是None）。 beta是移动平均线的系数，因为GANTrainertracks分别是发电机损失和批评损失。 gen_first = True表示训练从生成器开始（如果批评者为假，则为评论员）。 如果show_img = True，我们会在每个纪元的末尾显示生成的图像。

**switch**

`switch(gen_mode:bool=None)`

如果提供了gen_mode，则以所需模式切换模型。

如果gen_mode保留为None，则只需将模型置于其他模式（评论者是否处于生成器模式，反之亦然）。

**on_train_begin**

`on_train_begin(**kwargs)`

如有必要，为生成器和批评者创建优化器，初始化平滑器。

**on_epoch_begin**

`on_epoch_begin(epoch, **kwargs)`

如有必要，将评论家或发电机放回eval。

**on_batch_begin**

`on_batch_begin(last_input, last_target, **kwargs)`

如果不是None，用self.clip钳制权重，返回正确的输入。

**on_backward_begin**

`on_backward_begin(last_loss, last_output, **kwargs)`

将last_loss记录在正确的列表中。

**on_epoch_end**

`on_epoch_end(pbar, epoch, last_metrics, **kwargs)`

将各种损失放入录像机并显示样本图像。

**on_train_end**

`on_train_end(**kwargs)`

切换到发电机模式以显示结果。

---

## Specific modules

**class GANModule**

`GANModule(generator:Module=None, critic:Module=None, gen_mode:bool=False) :: PrePostInitMeta :: Module`

包围发电机和批评者创建一个GAN。

如果gen_mode保留为None，则只需将模型置于其他模式（评论者是否处于生成器模式，反之亦然）。

**switch**

`switch(gen_mode:bool=None)`

如果是gen_mode，则将模型置于生成器模式，否则为批评模式。

**class GANLoss**

`GANLoss(loss_funcG:Callable, loss_funcC:Callable, gan_model:GANModule) :: PrePostInitMeta :: GANModule`

包含loss_funcC（对于评论家）和loss_funcG（对于生成器）的包装。

**class AdaptiveLoss**

`AdaptiveLoss(crit) :: PrePostInitMeta :: Module`

在应用暴击之前展开目标以匹配输出大小。

**accuracy_thresh_expand**

`accuracy_thresh_expand(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True) → Rank0Tensor`

将y_true扩展为y_pred的大小后计算准确性。

---

## Data Block API

**class NoisyItem**

`NoisyItem(noise_sz) :: ItemBase`

一个大小为noise_sz的随机ItemBase。

**class GANItemList**

`GANItemList(items, noise_sz:int=100, **kwargs) :: ImageList`

ItemList适用于GAN。

输入将是noise_sz的NoisyItem，而目标的默认类是ImageList。

**show_xys**

`show_xys(xs, ys, imgsize:int=4, figsize:Optional[Tuple[int, int]]=None, **kwargs)`

在figsize图上显示ys（目标图像）。

**show_xyzs**

`show_xyzs(xs, ys, zs, imgsize:int=4, figsize:Optional[Tuple[int, int]]=None, **kwargs)`

在figsize图上显示zs（生成的图像）。
